# 🖼️ Estandarización y Preprocesamiento de Imágenes

Este notebook realiza la **estandarización y preprocesamiento** de todas las imágenes del dataset (OASIS + Kaggle) para garantizar un formato consistente antes del feature engineering.

**Este notebook utiliza los mismos métodos de preprocesamiento que el notebook 4** para garantizar consistencia en todo el pipeline.

## 🎯 Objetivos

1. **Cargar todas las imágenes** en múltiples formatos (PNG, JPG, JPEG)
2. **Normalizar las intensidades** usando z-score sobre voxeles no cero
3. **Recortar al brain bounding box** para eliminar el fondo
4. **Estandarizar el tamaño** a 224×224 píxeles
5. **Guardar las imágenes estandarizadas** en formato PNG para uso posterior
6. **Mantener la estructura de directorios** (CN, MCI, AD)

## 📋 Proceso

- **Entrada**: Imágenes en `data/processed/OASIS_2D/` (formatos: PNG, JPG, JPEG)
- **Procesamiento** (mismo flujo que notebook 4):
  1. Conversión a escala de grises
  2. Normalización z-score sobre voxeles no cero (`zscore_normalize`)
  3. Recorte a brain bounding box (`crop_brain`)
  4. Redimensionamiento a 224×224 (`resize224`)
- **Salida**: Imágenes estandarizadas guardadas en el mismo directorio (formato PNG, 224×224, normalizadas)


## 0. Imports y configuración


In [ ]:
import os
import numpy as np
from PIL import Image
from glob import glob
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path(r"C:\Users\mmera\OneDrive\Escritorio\ProyectoMineria\proyectoMineria")
IMG_DIR = BASE_DIR / 'data' / 'processed' / 'OASIS_2D'
TARGET_SIZE = (224, 224)
CLASSES = ['CN', 'MCI', 'AD']

print(f"📁 Directorio de imágenes: {IMG_DIR}")
print(f"📐 Tamaño objetivo: {TARGET_SIZE}")
print(f"📊 Clases: {CLASSES}")


## 1. Funciones de preprocesamiento

### 1.1. Normalización z-score
Normaliza la imagen usando z-score sobre voxeles no cero (solo el cerebro, no el fondo).

**Nota:** Esta función es equivalente a `zscore_nonzero` del notebook 4.


In [ ]:
def zscore_normalize(img: np.ndarray) -> np.ndarray:
    """
    Normaliza la imagen usando z-score sobre voxeles no cero.
    Equivalente a `zscore_nonzero` del notebook 4.
    
    Args:
        img: Imagen en escala de grises (2D array)
    
    Returns:
        Imagen normalizada
    """
    mask = img > 0
    if not np.any(mask):
        return img.astype(np.float32)
    
    vals = img[mask]
    mu = vals.mean()
    sigma = vals.std() + 1e-6  # Evitar división por cero
    
    img_normalized = (img - mu) / sigma
    return img_normalized.astype(np.float32)

# Prueba rápida
test_img = np.array([[0, 0, 0], [0, 100, 150], [0, 120, 200]], dtype=np.float32)
test_normalized = zscore_normalize(test_img)
print("✓ Función de normalización z-score definida (equivalente a zscore_nonzero del notebook 4)")
print(f"  Ejemplo: media={test_normalized[test_img > 0].mean():.2f}, std={test_normalized[test_img > 0].std():.2f}")


### 1.2. Recorte a Brain Bounding Box
Recorta la imagen al bounding box del cerebro (voxeles > 0) para eliminar el fondo y enfocarse solo en el cerebro.

**Nota:** Esta función es equivalente a `crop_brain` del notebook 4.


In [ ]:
def crop_brain(img: np.ndarray) -> np.ndarray:
    """
    Recorta la imagen al bounding box de voxeles > 0 (solo el cerebro, sin fondo).
    Si no hay voxeles > 0, regresa la imagen original.
    Equivalente a `crop_brain` del notebook 4.
    
    Args:
        img: Imagen en escala de grises (2D array)
    
    Returns:
        Imagen recortada al bounding box del cerebro
    """
    mask = img > 0
    if not np.any(mask):
        return img
    ys, xs = np.where(mask)
    y_min, y_max = ys.min(), ys.max()
    x_min, x_max = xs.min(), xs.max()
    return img[y_min:y_max+1, x_min:x_max+1]

# Prueba rápida
test_cropped = crop_brain(test_normalized)
print("✓ Función de recorte (crop_brain) definida (equivalente a crop_brain del notebook 4)")
print(f"  Tamaño original: {test_normalized.shape}")
print(f"  Tamaño después de crop: {test_cropped.shape}")


### 1.3. Redimensionamiento
Redimensiona la imagen a 224×224 usando interpolación bilineal.

**Nota:** Esta función es equivalente a `resize224` del notebook 4.


In [ ]:
def resize224(img: np.ndarray) -> np.ndarray:
    """
    Redimensiona la imagen a 224×224 usando interpolación bilineal.
    Equivalente a `resize224` del notebook 4.
    
    Args:
        img: Imagen en escala de grises (2D array)
    
    Returns:
        Imagen redimensionada a 224×224
    """
    pil = Image.fromarray(img.astype(np.float32))
    pil_resized = pil.resize((224, 224), Image.BILINEAR)
    return np.array(pil_resized, dtype=np.float32)

# Prueba rápida
test_resized = resize224(test_cropped)
print("✓ Función de redimensionamiento (resize224) definida (equivalente a resize224 del notebook 4)")
print(f"  Tamaño después de resize: {test_resized.shape}")


### 1.4. Función completa de preprocesamiento
Combina todas las operaciones siguiendo el mismo flujo que el notebook 4:
1. Carga de imagen
2. Normalización z-score sobre voxeles no cero
3. Recorte a brain bounding box
4. Redimensionamiento a 224×224


In [ ]:
def preprocess_image(image_path: Path, target_size: tuple = (224, 224)) -> np.ndarray:
    """
    Preprocesa una imagen completa siguiendo el mismo flujo que el notebook 4:
    1. Carga de imagen
    2. Normalización z-score sobre voxeles no cero (zscore_normalize)
    3. Recorte a brain bounding box (crop_brain)
    4. Redimensionamiento a 224×224 (resize224)
    
    Args:
        image_path: Ruta a la imagen
        target_size: Tamaño objetivo (por defecto 224×224, se ignora si se usa resize224)
    
    Returns:
        Imagen preprocesada (224×224, normalizada)
    """
    try:
        # 1. Cargar imagen y convertir a escala de grises
        img = np.array(Image.open(image_path).convert('L'), dtype=np.float32)
        
        # 2. Normalizar usando z-score sobre voxeles no cero
        img_normalized = zscore_normalize(img)
        
        # 3. Recortar al bounding box del cerebro
        img_cropped = crop_brain(img_normalized)
        
        # 4. Redimensionar a 224×224
        img_resized = resize224(img_cropped)
        
        return img_resized
    except Exception as e:
        print(f"⚠️  Error procesando {image_path}: {e}")
        return None

print("✓ Función completa de preprocesamiento definida")
print("  Flujo: carga → zscore_normalize → crop_brain → resize224")
print("  (Mismo flujo que el notebook 4)")


In [ ]:
# Buscar todas las imágenes en múltiples formatos
image_paths = {}
for c in CLASSES:
    class_dir = IMG_DIR / c
    if not class_dir.exists():
        print(f"⚠️  Directorio {class_dir} no existe")
        image_paths[c] = []
        continue
    
    all_images = []
    for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
        all_images.extend(list(class_dir.glob(ext)))
    
    image_paths[c] = sorted(all_images)
    print(f"📊 {c}: {len(image_paths[c])} imágenes encontradas")

total_images = sum(len(paths) for paths in image_paths.values())
print(f"\n📈 Total de imágenes: {total_images}")


## 3. Procesamiento y guardado de imágenes estandarizadas

Procesamos todas las imágenes y las guardamos en formato PNG estandarizado.


In [ ]:
def save_preprocessed_image(img: np.ndarray, output_path: Path):
    """
    Guarda una imagen preprocesada en formato PNG.
    Convierte los valores normalizados a rango 0-255 para guardar.
    
    Args:
        img: Imagen preprocesada (normalizada, puede tener valores negativos)
        output_path: Ruta donde guardar la imagen
    """
    # Convertir de z-score a rango 0-255
    # Primero normalizar a rango [0, 1] usando min-max
    img_min = img.min()
    img_max = img.max()
    if img_max > img_min:
        img_scaled = (img - img_min) / (img_max - img_min)
    else:
        img_scaled = np.zeros_like(img)
    
    # Convertir a uint8
    img_uint8 = (img_scaled * 255).astype(np.uint8)
    
    # Guardar como PNG
    Image.fromarray(img_uint8).save(output_path, 'PNG')

print("✓ Función de guardado definida")


In [ ]:
# Procesar todas las imágenes
stats = {c: {'processed': 0, 'errors': 0, 'skipped': 0} for c in CLASSES}

for c in CLASSES:
    print(f"\n🔄 Procesando clase: {c}")
    class_dir = IMG_DIR / c
    
    for img_path in tqdm(image_paths[c], desc=f"  {c}"):
        try:
            # Preprocesar imagen
            img_preprocessed = preprocess_image(img_path, TARGET_SIZE)
            
            if img_preprocessed is None:
                stats[c]['errors'] += 1
                continue
            
            # Determinar nombre de salida (siempre PNG)
            if img_path.suffix.lower() in ['.jpg', '.jpeg']:
                # Si es JPG, guardar como PNG con mismo nombre base
                output_path = img_path.with_suffix('.png')
                # Si ya existe un PNG con ese nombre, agregar sufijo
                if output_path.exists() and output_path != img_path:
                    output_path = img_path.parent / f"{img_path.stem}_std.png"
            else:
                # Si ya es PNG, sobrescribir o crear copia
                output_path = img_path.parent / f"{img_path.stem}_std.png"
            
            # Guardar imagen preprocesada
            save_preprocessed_image(img_preprocessed, output_path)
            stats[c]['processed'] += 1
            
        except Exception as e:
            print(f"  ⚠️  Error en {img_path.name}: {e}")
            stats[c]['errors'] += 1

print("\n" + "="*50)
print("📊 Resumen de procesamiento:")
print("="*50)
for c in CLASSES:
    print(f"{c}:")
    print(f"  ✅ Procesadas: {stats[c]['processed']}")
    print(f"  ❌ Errores: {stats[c]['errors']}")
    print(f"  ⏭️  Omitidas: {stats[c]['skipped']}")


## 4. Verificación: Visualizar imágenes estandarizadas

Verificamos que las imágenes se hayan procesado correctamente visualizando algunas muestras.


In [ ]:
import matplotlib.pyplot as plt

# Buscar imágenes estandarizadas (con sufijo _std.png)
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle('Imágenes Estandarizadas (224×224, Normalizadas)', fontsize=16)

for idx, c in enumerate(CLASSES):
    class_dir = IMG_DIR / c
    std_images = list(class_dir.glob('*_std.png'))
    
    if not std_images:
        # Si no hay _std.png, buscar cualquier PNG
        std_images = list(class_dir.glob('*.png'))[:3]
    
    for j in range(3):
        ax = axes[idx, j]
        if j < len(std_images):
            img = np.array(Image.open(std_images[j]).convert('L'))
            ax.imshow(img, cmap='gray')
            ax.set_title(f'{c} - {std_images[j].name[:30]}', fontsize=8)
        else:
            ax.axis('off')
        ax.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualización completada")


## 5. Estadísticas finales

Verificamos el estado final del dataset estandarizado.


In [ ]:
# Contar imágenes estandarizadas finales
final_stats = {}
for c in CLASSES:
    class_dir = IMG_DIR / c
    if class_dir.exists():
        # Contar todas las imágenes PNG (incluyendo _std.png)
        png_images = list(class_dir.glob('*.png'))
        final_stats[c] = len(png_images)
    else:
        final_stats[c] = 0

print("="*50)
print("📊 Estadísticas finales del dataset estandarizado:")
print("="*50)
for c in CLASSES:
    print(f"{c}: {final_stats[c]} imágenes PNG")

total_final = sum(final_stats.values())
print(f"\n📈 Total: {total_final} imágenes estandarizadas")
print(f"📐 Tamaño: {TARGET_SIZE[0]}×{TARGET_SIZE[1]} píxeles")
print(f"📁 Formato: PNG (escala de grises, normalizadas)")
print("\n✅ Proceso de estandarización completado")
